In [ ]:
import pandas as pd
from google_sheets import get_spreadsheet_data

## Fetch the data from the datasheet

In [ ]:
df = get_spreadsheet_data()
easy_apply = df[df['Comments'].str.contains('LinkedIn Easy Apply', na=False)]
active_applications = df[df["Offer"] != "No"]

In [6]:
active_applications['Last Communication Date'] = pd.to_datetime(active_applications['Last Communication Date'])
needs_followup = active_applications[(pd.Timestamp.now() - active_applications['Last Communication Date']) > pd.Timedelta(days=7)]
